In [1]:
import pickle
import numpy as np
from tensorflow.keras.utils import to_categorical

def load_cifar10_batch(file_path):
    with open(file_path, 'rb') as file:
        batch = pickle.load(file, encoding='bytes')
    return batch

def load_cifar10_data(folder_path):
    train_data = []
    train_labels = []

    for i in range(1, 6):
        batch_file = f"{folder_path}/data_batch_{i}"
        batch = load_cifar10_batch(batch_file)
        train_data.append(batch[b'data'])
        train_labels.extend(batch[b'labels'])

    test_batch_file = f"{folder_path}/test_batch"
    test_batch = load_cifar10_batch(test_batch_file)
    test_data = test_batch[b'data']
    test_labels = test_batch[b'labels']

    train_data = np.vstack(train_data)
    train_labels = np.array(train_labels)
    test_labels = np.array(test_labels)

    return train_data, train_labels, test_data, test_labels

def preprocess_data(train_data, train_labels, test_data, test_labels):
    train_data = train_data.reshape(-1, 3, 32, 32).transpose(0, 2, 3, 1)
    test_data = test_data.reshape(-1, 3, 32, 32).transpose(0, 2, 3, 1)

    train_labels_onehot = to_categorical(train_labels)
    test_labels_onehot = to_categorical(test_labels)

    return train_data, train_labels_onehot, test_data, test_labels_onehot

cifar10_folder = 'cifar-10-batches-py'

train_data, train_labels, test_data, test_labels = load_cifar10_data(cifar10_folder)

x_train, y_train, x_test, y_test = preprocess_data(
    train_data, train_labels, test_data, test_labels
)

print("Train Data Shape:", x_train.shape)
print("Train Labels Shape:", y_train.shape)
print("Test Data Shape:", x_test.shape)
print("Test Labels Shape:", y_test.shape)

Train Data Shape: (50000, 32, 32, 3)
Train Labels Shape: (50000, 10)
Test Data Shape: (10000, 32, 32, 3)
Test Labels Shape: (10000, 10)


In [2]:
import tensorflow_hub as hub
from tqdm import tqdm
import tensorflow as tf

supres_model = hub.load('https://tfhub.dev/captain-pool/esrgan-tf2/1')
x_train_superres = supres_model(tf.cast(x_train[0:100], tf.float32), training=False).numpy().astype('uint8')
for i in tqdm(range(100, x_train.shape[0], 100)):
    result = supres_model(tf.cast(x_train[i:i+100], tf.float32), training=False).numpy().astype('uint8')
    x_train_superres = np.concatenate((x_train_superres, result), axis=0)

x_test_superres = supres_model(tf.cast(x_test[0:100], tf.float32), training=False).numpy().astype('uint8')
for i in tqdm(range(100, x_test.shape[0], 100)):
    result = supres_model(tf.cast(x_test[i:i+100], tf.float32), training=False).numpy().astype('uint8')
    x_test_superres = np.concatenate((x_test_superres, result), axis=0)

np.save('x_train_superres.npy', x_train_superres)
np.save('x_test_superres.npy', x_test_superres)

100%|██████████| 99/99 [00:48<00:00,  2.05it/s]
